### pptx to text to speech to pptx

pptxファイルからメモの文字列を抽出し、音声データとして埋め込む。

Ubuntu 20.4LTS(WSL)にAnaconda3をインストールしている。

Anacondaとffmpegのインストールはちょっと時間がかかる。

## WSL側の追加準備
WSLでffmpegを入れておくこと。  
sudo apt-get update  
sudo apt-get install ffmpeg  

## Pythonがわのパッケージ追加
以下しておくこと。  
pipでも、conda installでもお好みでどうぞ。

pip install google-api-python-client  
pip install google-cloud-datastore  
pip install --upgrade google-cloud-texttospeech  
pip install python-pptx

pip install pydub

In [1]:
# import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
from google.cloud import texttospeech
from google.oauth2 import service_account
from pptx import Presentation, shapes
from pptx.util import Cm
import os
import pydub
import numpy as np
import wave  # wave
import struct  # structpack


### ファイルパス関連
セキュリティーjsonファイルが必要である。  
以下のjsonファイルがキーファイルである。  
以下などを参照のこと  

https://blog.apar.jp/web/9893/  

ここでの「認証情報の作成」として記載されているjsonファイルが必要。


https://www.marketechlabo.com/python-google-auth/

https://co-lab.contents.ne.jp/20181019-1902


In [3]:
##jsonファイルのパスを次のセルで指定しているが、お見せしたくないので以下のような指定だと思ってもらいたい。
f_json = "./jsons_path/keyfilename.json"


In [5]:
## input fileのパス
f_pptx = "./pptxfiles/test01.pptx"

## アウトプットパス
f_out = f_pptx.replace(".pptx", "a.pptx")

# 処理フォルダ
dir_temp = "./temp/"
if os.path.exists(dir_temp):
    print("dir exsist.")
else:
    print("dir not exists")
    os.makedirs(dir_temp)

dir exsist.


ファイルの読み込み

In [6]:
prs = Presentation(f_pptx)


In [7]:
## 32khzで無音音声を作る。t秒。

def createWavFile_lin(f,t):
    Fs = 32*1000
    
    # もともとは正弦波で音を作る関数をいじっている。
    # 無駄に冗長なのは勘弁
    ch1 = 440 
    
    #無音にするための倍数
    r1 =0
    
    
    scale = 10**(-3/10)  # 音量
    N = Fs*t  # サンプル数
    filename = f



    # 正弦波の配列を生成
    sw1 = np.array([np.sin(2*np.pi*ch1*n/Fs)
                                 for n in np.arange(N)])
    sw = r1*sw1 
    
    if np.max(sw) == 0:
        divmax = 1
    else:
        divmax = np.max(sw)
    
    sw = scale*sw /divmax
    sw = np.nan_to_num(sw, nan=0)

    # 16ビットに変換
    waveL = np.array([int(x * 32767.0) for x in sw])
    waveR = np.array([int(x * 32767.0) for x in sw])

    # ステレオ形式のwavのデータ構造に合わせ込む
    quantizeWave = np.array([[waveL[i], waveR[i]]
                             for i in range(int(N))]).flatten()

    # バイナリ化
    binwave = struct.pack("h" * len(waveL)*2, *quantizeWave)
    w = wave.Wave_write(filename)
    # 参考：https://docs.python.org/ja/3/library/wave.html
    p = (2, 2, Fs, len(binwave), 'NONE', 'not compressed')

    w.setparams(p)
    w.writeframes(binwave)
    w.close()



無音音声を作る。  
以下当たり前だが何も聞こえない。

In [8]:
import IPython

##0.5秒の空白音声を作っておく。
f_emp = "./marking_emp.wav"
t =0.5
createWavFile_lin(f=f_emp,t=t)
IPython.display.Audio(f_emp)

スライドごとに:en:/:ne:で挟まれた文字列を抽出する。  
目的の文字列が取れているだろうか？

In [9]:
d_txt = dict()
for i, sld in enumerate(prs.slides, start=1):
    print(f'-- {i} --')
    nt = sld.notes_slide.notes_text_frame.text
    if ":en:" in nt:
        nt1 = nt.split(":en:")[1]
        nt2 = nt1.split(":ne:")[0]
        nt3 = nt2.replace("  "," ")
        print(nt3)
        d_txt[i] = {
            "note":nt,
            "txt":nt3
        }

-- 1 --

Good morning.
I am John Due from JD university.
Today, I will talk about a method to speak English.

-- 2 --

This is last.



In [10]:
# d_txt

## API
ここで初めてapiの作業  
キーjsonを使ってcredentialsを作る。  
これを使ってclientを作る。

In [12]:
credentials = service_account.Credentials.from_service_account_file(f_json)
client = texttospeech.TextToSpeechClient(credentials=credentials)

## 音声の設定  
この辺を参照されたい。  
https://cloud.google.com/text-to-speech?hl=ja

In [13]:
voice = texttospeech.types.VoiceSelectionParams(
  language_code='en-US',
  name='en-US-Wavenet-F',
  ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)

audio_config = texttospeech.types.AudioConfig(
  audio_encoding=texttospeech.enums.AudioEncoding.MP3,
  pitch = 0,
    speaking_rate =1.0
  )

In [14]:
#google.cloud.texttospeechの古い方ではこの様に指定していた

# voice = texttospeech.VoiceSelectionParams(
#   language_code='en-US',
#   name='en-US-Wavenet-F',
#   ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)

# audio_config = texttospeech.AudioConfig(
#   audio_encoding=texttospeech.AudioEncoding.MP3,
#   pitch = 0,
#     speaking_rate =1.0
#   )

In [33]:
# import ffmpeg

In [16]:
#一旦このファイルに書き出す。
fn0 = "./temp/temp1.mp3"

#先程作った無音ファイル
sound0 = pydub.AudioSegment.from_wav(f_emp)

for k, v in d_txt.items():
    fn = "./temp/temp_sound_"+str(k)+".wav"
    print(k,fn, v["txt"])
    
    # synthesis_input = texttospeech.SynthesisInput(text = v["txt"])
    # response = client.synthesize_speech(input=synthesis_input, voice=voice, audio_config=audio_config)
    synthesis_input = texttospeech.types.SynthesisInput(text = v["txt"])
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open(fn0, 'wb') as out:
        out.write(response.audio_content)
        # print(f'Audio content written to file {fn0}')
    s1 = pydub.AudioSegment.from_mp3(fn0)
    
    #ここでガッチャンコ
    s01 = sound0 + s1
    s01.export(fn, format="wav")
    
    #ここで保存したファイルを再度挿入
    #位置はスライドに応じて適当に調整されたい。
    prs.slides[k -1].shapes.add_movie(fn, left=Cm(18), top=Cm(0), width=Cm(2), height=Cm(2))
#     break


1 ./temp/temp_sound_1.wav 
Good morning.
I am John Due from JD university.
Today, I will talk about a method to speak English.

2 ./temp/temp_sound_2.wav 
This is last.



In [17]:
prs.save(f_out)